<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [23]:
import pandas as pd
import unicodedata
import random
from train import training
from util import load_data, evaluate,  df_tolist
from sklearn.utils import shuffle

In [24]:
# change to dataframe to Spacy format
# wikidata featured articles scraped text
df = pd.read_csv('E:/Sharpest_Mind/WikipediaCitation/src/Notebooks/Sentences_traindata.csv', encoding = 'utf8')
df['Statements'] = (df['Statements'].map(lambda x: unicodedata.normalize('NFKD', str(x))))
df = df[(df['Statements'].str.split().str.len() >10)].reset_index(drop=True)
df['Statements'] = df['Statements'].astype(str)
df =shuffle(df)
df = df.drop_duplicates()
print(df.shape)
print(df.label.value_counts())
df.head()

(36340, 2)
1    19107
0    17233
Name: label, dtype: int64


,Statements,label
5177,"For the rest of the year, it serves as The Pla...",1
23939,"where i is the imaginary unit, or can alternat...",0
26003,"The next day, Maggie causes distractions when ...",0
15498,"First published in 1924, it soon sold out, res...",1
6041,Aurelian spared the city and stationed a garri...,1


In [25]:
df1 = df[df['label']==1]
df1 =shuffle(df1)
df2 = df[df['label']==0]
df2 =shuffle(df2)
df3=pd.concat([df1[:10000],df2[:10000]])
df3.label.value_counts()

1    10000
0    10000
Name: label, dtype: int64

In [26]:
# Processing the final format of training and validation data
# Calling the load_data() function 
(train_texts, train_cats), (dev_texts, dev_cats) = load_data(df3,split=0.8)
train_data = list(zip(train_texts,[{'cats': cats} for cats in train_cats]))
print(train_data[:1])
dev_data = list(zip(dev_texts,[{'cats': cats} for cats in dev_cats]))
print(dev_data[:1])


[("Waugh was the son of a publisher, educated at Lancing College and then at Hertford College, Oxford. He worked briefly as a schoolmaster before he became a full-time writer. As a young man, he acquired many fashionable and aristocratic friends and developed a taste for country house society. He travelled extensively in the 1930s, often as a special newspaper correspondent; he reported from Abyssinia at the time of the 1935 Italian invasion. He served in the British armed forces throughout the Second World War, first in the Royal Marines and then in the Royal Horse Guards. He was a perceptive writer who used the experiences and the wide range of people whom he encountered in his works of fiction, generally to humorous effect. Waugh's detachment was such that he fictionalised his own mental breakdown which occurred in the early 1950s.", {'cats': {'POSITIVE': False, 'NEGATIVE': True}})]
[("The parish accepts the diocese's guidance on permitting baptism and marriage in church after a div

E:\Sharpest_Mind\WikipediaCitation\src\Notebooks\util.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tuples'] = df.apply(lambda row: (row[df.columns[0]], row[df.columns[1]]), axis=1)


In [27]:

# change to dataframe to Spacy format
# wikidata citation needed text
df4 = pd.read_csv('E:/Sharpest_Mind/WikipediaCitation/data/processed/csv_files/sort/Citation_needed_wiki.csv', encoding = 'ISO-8859-1')
df4 = shuffle(df4)
df4_1 = df4[df4['label']==1] 
df4_0 = df4[df4['label']==0]
df4_test = pd.concat([df4_1[:2000],df4_0[:2000]])
test1 =df_tolist(df4_test)
test_texts1, test_cats1 = zip(*test1)
test_cats1 = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)} for y in test_cats1]
test_data1 = list(zip(test_texts1,[{'cats': cats} for cats in test_cats1]))
print(len(test_data1))

4000


In [28]:

# wikidata random articles -biography section
df5 = pd.read_csv('E:/Sharpest_Mind/WikipediaCitation/data/processed/csv_files/sort/wiki_data_test.csv', encoding = 'ISO-8859-1')
df5 = shuffle(df5)
df5_1 = df5[df5['Label']==1] 
df5_0 = df5[df5['Label']==0]
df5_test = pd.concat([df5_1[8000:],df5_0[8000:]])
test2 =df_tolist(df5_test)

random.shuffle(test2)

test_texts2, test_cats2 = zip(*test2)
test_cats2 = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)} for y in test_cats2]
test_data2 = list(zip(test_texts2,[{'cats': cats} for cats in test_cats2]))
print(len(test_data2))

3980


In [29]:

# wikidata random articles -biography section
df6 = pd.read_csv('E:/Sharpest_Mind/WikipediaCitation/data/processed/csv_files/sort/random_monument.csv', encoding = 'ISO-8859-1')
df6 = shuffle(df6)
df6_1 = df6[df6['label']==1] 
df6_0 = df6[df6['label']==0]
df6_test = pd.concat([df6_1[8000:],df6_0[8000:]])
test3 =df_tolist(df6_test)

random.shuffle(test3)

test_texts3, test_cats3 = zip(*test3)
test_cats3 = [{"POSITIVE": bool(y), "NEGATIVE": not bool(y)} for y in test_cats3]
test_data3 = list(zip(test_texts2,[{'cats': cats} for cats in test_cats3]))
print(len(test_data3))

3980


In [30]:
print(len(train_data))
print(len(test_data1))
print(len(test_data2))
print(len(dev_data))

16000
4000
3980
4000


In [ ]:
# Model 1 - CNN, L2 =2e-4
train_results1, dev_results1, test_results1 = training(train_texts, train_cats, dev_texts, dev_cats, test_texts3, test_cats3, L2 = 2e-5, 
                                                     learn_rate = 0.001, n_iter = 3, output_dir='model_artifactnewdatatest1L2-2e-5')

Training the model...
LOSS 	A_train	A_dev	A_test	  P  	  R  	  F  


In [ ]:
### Model 2 - Ensemble, L2 0.0001 
train_results2, dev_results2, test_results2 = training(train_texts, train_cats, dev_texts, dev_cats, test_texts2, test_cats2, L2 = 2e-5, 
                                                     learn_rate = 0.001, n_iter=3,  output_dir='model_artifactnewtest2-2e-5')

In [ ]:
# Model 3
train_restul3, dev_results3, test_result3 = training(train_texts, train_cats, dev_texts, dev_cats, test_texts1, test_cats1, L2 = 2e-4, 
                                                     learn_rate = 0.001, output_dir='model_artifactwikitest')

In [ ]:
import matplotlib.pyplot as plt

accuracy_train = [(d['textcat_a']) for d in train_results1]
accuracy_dev = [(d['textcat_a']) for d in dev_results1]
accuracy_test = [(d['textcat_a']) for d in test_results1]
accuracy_train1 = [(d['textcat_a']) for d in train_restul3]
accuracy_dev1 = [(d['textcat_a']) for d in dev_results3]
accuracy_test1 = [(d['textcat_a']) for d in test_result3]
epochs = range(0,5)
plt.plot(epochs, accuracy_train, 'g', label='Train model1')
plt.plot(epochs, accuracy_dev, 'k', label='Val model1')
plt.plot(epochs, accuracy_test, 'b', label='Test model1')
plt.plot(epochs, accuracy_train1, 'r', label='Train model2')
plt.plot(epochs, accuracy_dev1, 'c', label='Vali model2')
plt.plot(epochs, accuracy_test1, 'y', label='Test model2')
plt.title('Training and Validation Accuracy Model 2 =simple_cnn and Model1 = Ensemble')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

accuracy_train = [(d['textcat_a']) for d in train_restul3]
accuracy_dev = [(d['textcat_a']) for d in dev_results3]
accuracy_test = [(d['textcat_a']) for d in test_result3]

epochs = range(0,5)
plt.plot(epochs, accuracy_train, 'g', label='Training accuracy_model1')
plt.plot(epochs, accuracy_dev, 'k', label='Validation accuracy_model1')
plt.plot(epochs, accuracy_test, 'b', label='Testing accuracy_model1')

plt.title('Training and Validation Accuracy LR2=0.1 Model1 = Ensemble, Model 2 =simple_cnn')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

precision_test1 = [(d['textcat_p']) for d in test_results1]
precision_test2 = [(d['textcat_p']) for d in test_results2]
precision_test3 = [(d['textcat_p']) for d in test_result3]
#precision_test4 = [(d['textcat_p']) for d in test_result4]

epochs = range(0,5)
plt.plot(epochs, precision_test1, 'g', label='1 - LR :2e-4&cnn')
plt.plot(epochs, precision_test2, 'b', label='2 - LR :2e-4&cnn')
plt.plot(epochs, precision_test3, 'r', label='3')
#plt.plot(epochs, precision_test4, 'c', label='4')
plt.title('Training and Validation precision 1 = =simple_cnn')
plt.xlabel('Epochs')
plt.ylabel('precision')
plt.legend()
plt.show()

In [ ]:
# Testing the model = https://en.wikipedia.org/wiki/Che_Guevara - positive sentence
test_text= "Such positions also allowed him to play a central role in training the militia forces who repelled the Bay of Pigs Invasion and bringing the Soviet nuclear-armed ballistic missiles to Cuba which precipitated the 1962 Cuban Missile Crisis. "
doc=loaded_model(test_text)
doc.cats

In [ ]:
import spacy


In [ ]:
# Testing the model - negative sentence

loaded_model = spacy.load('E:/Sharpest_Mind/WikipediaCitation/notebooks/model_artifactwikitest2-2e-3')
test_text="Later, during the Age of Exploration, maps became progressively more accurate for navigation needs and were often sprinkled with sketches and drawings such as sailing ships showing the direction of trade winds, little trees and mounds to represent forests and mountains and of course, plenty of sea creatures and exotic natives much of them imaginary. As the need for geographical accuracy increased, these illustrations gradually slipped off the map and onto the borders and eventually disappeared altogether in the wake of modern scientific cartography."
doc=loaded_model(test_text)
doc.cats
#spacy.displacy.render(doc, style='ent')

In [ ]:
#positive sentence 

test_text1 = 'Anthropomorphic maps date back to when Sebastian Münster used a queen to depict Europe in 1570.'
doc=loaded_model(test_text1)
doc.cats